<a href="https://colab.research.google.com/github/dibakar75/DeepCNNClassifierProject/blob/main/insurance_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.layers import Dense, Dropout

In [3]:
df = pd.read_csv('/content/sample_data/insurance_pred.csv')

In [4]:
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
X = df.iloc[:, :6]
y = df.iloc[:,6]

In [7]:
#create dummy variables
Sex = pd.get_dummies(df["sex"], drop_first = 'True')
Region = pd.get_dummies(df["region"], drop_first = 'True')
Smoker = pd.get_dummies(df["smoker"], drop_first = 'True')

In [8]:
df = pd.concat([df, Sex, Region, Smoker], axis=1)

In [9]:
df

,age,sex,bmi,children,smoker,region,expenses,male,northwest,southeast,southwest,yes
0,19,female,27.9,0,yes,southwest,16884.92,0,0,0,1,1
1,18,male,33.8,1,no,southeast,1725.55,1,0,1,0,0
2,28,male,33.0,3,no,southeast,4449.46,1,0,1,0,0
3,33,male,22.7,0,no,northwest,21984.47,1,1,0,0,0
4,32,male,28.9,0,no,northwest,3866.86,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55,1,1,0,0,0
1334,18,female,31.9,0,no,northeast,2205.98,0,0,0,0,0
1335,18,female,36.9,0,no,southeast,1629.83,0,0,1,0,0
1336,21,female,25.8,0,no,southwest,2007.95,0,0,0,1,0


In [10]:
df = df.drop(['sex', 'region','smoker'], axis=1)

In [11]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
print(f"Rows in train_set : {len(train_set)}\nRows in test_set : {len(test_set)}")

Rows in train_set : 1070
Rows in test_set : 268


In [12]:
#shape of train and test
print(train_set.shape)
print(test_set.shape)

(1070, 9)
(268, 9)


In [13]:
#Data description
train_set.describe()

,age,bmi,children,expenses,male,northwest,southeast,southwest,yes
count,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000
mean,39.357009,30.562150,1.107477,13346.089869,0.512150,0.239252,0.264486,0.246729,0.205607
std,14.073960,6.043266,1.215983,12019.510759,0.500086,0.426827,0.441265,0.431309,0.404334
min,18.000000,16.000000,0.000000,1121.870000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,26.200000,0.000000,4897.665000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.500000,30.200000,1.000000,9575.440000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,51.000000,34.500000,2.000000,16746.655000,1.000000,0.000000,1.000000,0.000000,0.000000
max,64.000000,53.100000,5.000000,62592.870000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
ind_var = 'expenses'

In [15]:
# split the data into dependent and independent
x_train, y_train = train_set.drop(ind_var, axis=1), train_set[ind_var]
x_test, y_test = test_set.drop(ind_var, axis=1), test_set[ind_var]

In [16]:
# Feature Scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [18]:
## KERAS tuner

In [ ]:
import pandas as pd

In [22]:
msle = MeanSquaredLogarithmicError()


def model_builder(hp):
  model = tf.keras.Sequential()
  
  units1 = hp.Int('units1', min_value=25, max_value=512, step=32)
  units2 = hp.Int('units2', min_value=25, max_value=512, step=32)
  units3 = hp.Int('units3', min_value=25, max_value=512, step=32)
  model.add(Dense(units=units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

In [23]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    model_builder,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

In [25]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [26]:
tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks = [stop_early])

Trial 42 Complete [00h 00m 03s]
val_mean_squared_logarithmic_error: 0.17927835881710052

Best val_mean_squared_logarithmic_error So Far: 0.17503680288791656
Total elapsed time: 00h 02m 22s


In [27]:
##Finding the best hyper parameter
for p in ['units1','units2','units3','learning_rate']:
  print(p, tuner.get_best_hyperparameters()[0].get(p))

units1 89
units2 217
units3 505
learning_rate 0.01


In [28]:
##Model Evaluation
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [29]:
model = tuner.hypermodel.build(best_hps)

In [30]:
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
27/27 [==============================] - 1s 26ms/step - loss: 13.9988 - mean_squared_logarithmic_error: 13.8765 - val_loss: 0.7388 - val_mean_squared_logarithmic_error: 0.7385
Epoch 2/50
27/27 [==============================] - 0s 6ms/step - loss: 0.5672 - mean_squared_logarithmic_error: 0.5663 - val_loss: 0.5529 - val_mean_squared_logarithmic_error: 0.5531
Epoch 3/50
27/27 [==============================] - 0s 4ms/step - loss: 0.3769 - mean_squared_logarithmic_error: 0.3756 - val_loss: 0.2993 - val_mean_squared_logarithmic_error: 0.3030
Epoch 4/50
27/27 [==============================] - 0s 4ms/step - loss: 0.2516 - mean_squared_logarithmic_error: 0.2520 - val_loss: 0.2234 - val_mean_squared_logarithmic_error: 0.2289
Epoch 5/50
27/27 [==============================] - 0s 4ms/step - loss: 0.2047 - mean_squared_logarithmic_error: 0.2037 - val_loss: 0.1977 - val_mean_squared_logarithmic_error: 0.2027
Epoch 6/50
27/27 [==============================] - 0s 5ms/step - loss: 0.181

In [31]:
eval_result = model.evaluate(x_test, y_test)
print(eval_result)

9/9 [==============================] - 0s 3ms/step - loss: 0.1505 - mean_squared_logarithmic_error: 0.1429
[0.15048184990882874, 0.14285314083099365]


In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 89)                801       
                                                                 
 dense_5 (Dense)             (None, 217)               19530     
                                                                 
 dense_6 (Dense)             (None, 505)               110090    
                                                                 
 dense_7 (Dense)             (None, 1)                 506       
                                                                 
Total params: 130,927
Trainable params: 130,927
Non-trainable params: 0
_________________________________________________________________
